
### Connection to the database

In [1]:
# Import the necessary libraries
import csv
import sqlite3

# Connect to the database
connexion = sqlite3.connect("db/vivino.db")
cursor = connexion.cursor()

In [2]:
# Query the name of all tables of the database
cursor.execute("""
    SELECT name
    FROM sqlite_master
    WHERE type='table';
""")
for name, in cursor.fetchall():
    print(name)

countries
grapes
wineries
flavor_groups
keywords
regions
most_used_grapes_per_country
toplists
wines
vintages
keywords_wine
vintage_toplists_rankings



### Query 1

In [3]:
# A query to get the three most used grapes in the world
cursor.execute("""
    SELECT
        g.name AS grape_name,
        COUNT(DISTINCT m.country_code) AS countries_count
    FROM most_used_grapes_per_country AS m
    LEFT JOIN grapes AS g
        ON m.grape_id = g.id
    GROUP BY g.name
    ORDER BY countries_count DESC
    LIMIT 3
""")

cursor.fetchall()

[('Cabernet Sauvignon', 12), ('Merlot', 11), ('Chardonnay', 6)]

In [4]:
# Query the wine name containing one of the following string:
# 'Cabernet Sauvignon', 'Merlot', and 'Chardonnay'
cursor.execute("""
    SELECT
        w.name AS wine_name,
        v.year,
        CASE WHEN w.name LIKE '%Cabernet Sauvignon%' THEN 'Cabernet Sauvignon'
             WHEN w.name LIKE '%Merlot%' THEN 'Merlot'
             WHEN w.name LIKE '%Chardonnay%' THEN 'Chardonnay'
             END AS grape_name,
        w.ratings_average,
        w.ratings_count,
        r.name AS region,
        c.name AS country
    FROM wines AS w
    INNER JOIN vintages AS v ON w.id = v.wine_id
    LEFT JOIN regions AS r ON w.region_id = r.id
    LEFT JOIN countries AS c ON r.country_code = c.code
    WHERE w.name LIKE '%Cabernet Sauvignon%'
        OR w.name LIKE '%Merlot%'
        OR w.name LIKE '%Chardonnay%'
    ORDER BY w.ratings_average DESC, w.ratings_count DESC;
""")

data_01 = [[item[0] for item in cursor.description]]

for item in cursor.fetchall():
    data_01.append(list(item))


with open('result_01.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data_01)

### Query 2

In [5]:
# Query the wine name containing the string 'Cabernet Sauvignon'
cursor.execute("""
    SELECT
        w.name AS wine_name,
        v.year,
        w.ratings_average,
        w.ratings_count,
        (v.price_euros/v.bottle_volume_ml)*1000 AS liter_price_euros,
        c.name AS country
    FROM wines AS w
    INNER JOIN vintages AS v ON w.id = v.wine_id
    LEFT JOIN regions AS r ON w.region_id = r.id
    LEFT JOIN countries AS c ON r.country_code = c.code
    WHERE w.name LIKE '%Cabernet Sauvignon%'
    ORDER BY w.ratings_average DESC, w.ratings_count DESC;
""")

data_02 = [[item[0] for item in cursor.description]]

for item in cursor.fetchall():
    data_02.append(list(item))

with open('result_02.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data_02)


### Query 3

In [6]:
cursor.execute("""
    SELECT
        v.name AS vintage,
        v.year AS vintage_year,
        vtr.rank,
        vtr.previous_rank,
        REPLACE(t.name, 'Bestsellers in ', '') AS bestseller_region,
        c.name AS bestseller_country,
        price_euros,
        bottle_volume_ml
    FROM vintages AS v
    LEFT JOIN vintage_toplists_rankings AS vtr ON v.id = vtr.vintage_id
    LEFT JOIN toplists AS t ON vtr.top_list_id = t.id
    LEFT JOIN countries AS c ON t.country_code = c.code
    WHERE t.name LIKE 'Bestsellers%';
""")

data_03 = [[item[0] for item in cursor.description]]

for item in cursor.fetchall():
    data_03.append(list(item))

with open('result_03.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data_03)


### Query 4

In [7]:
cursor.execute("""
    SELECT
        v.name AS vintage,
        v.year AS vintage_year,
        vtr.rank,
        vtr.previous_rank,
        CAST(SUBSTR(t.name, 10, 4) AS int) AS award_year,
        SUBSTR(t.name, INSTR(t.name, ':')+2) AS award_name,
        price_euros,
        bottle_volume_ml
    FROM vintages AS v
    LEFT JOIN vintage_toplists_rankings AS vtr ON v.id = vtr.vintage_id
    LEFT JOIN toplists AS t ON vtr.top_list_id = t.id
    WHERE t.name LIKE 'Vivino%';
""")

data_04 = [[item[0] for item in cursor.description]]

for item in cursor.fetchall():
    data_04.append(list(item))

with open('result_04.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data_04)

In [8]:
def query_to_csv(query: str, csv_file_name: str):
    """
    Args:

    Returns:
    """
    connexion = sqlite3.connect("db/vivino.db")
    cursor = connexion.cursor()
    cursor.execute(query)
    data = [[item[0] for item in cursor.description]]
    for item in cursor.fetchall():
        data.append(list(item))
    
    with open(csv_file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)
    
    connexion.close()